# Using the Explorer
The explorer allows sweep exploration and saving sweeps to drop all within the browser.
- For ease, click the double-forward-arrow icon on top toolbar to run all the cells in this notebook and display the explorer. 
- You must have bokeh installed: `conda install bokeh`
- You must have ipyfilechooser installed: `pip install ipyfilechooser`
- To work in jupyter lab please follow guide here: https://docs.bokeh.org/en/latest/docs/user_guide/jupyter.html

In [1]:
from ipyfilechooser import FileChooser
from IPython.display import display
import ipywidgets as widgets
import os
import pandas as pd
import clamp_ephys
import numpy as np
from bokeh.io import output_notebook, show, push_notebook
from bokeh.plotting import figure
output_notebook()

/home/nate/miniconda3/envs/lab/lib/python3.7/site-packages/elephant/pandas_bridge.py:22: DeprecationWarning: pandas_bridge module will be removed in Elephant v0.8.x
  DeprecationWarning)


Loading BokehJS ...

In [2]:
lowpass_freq_ = widgets.IntText(value=500, description='Lowpass Filter Freq (Hz):', layout=widgets.Layout(width='150px'))
stim_time_ = widgets.IntText(value=500, description='Stim onset time (ms):', layout=widgets.Layout(width='150px'))
post_stim_ = widgets.IntText(value=250, description='Stim window length (ms):', layout=widgets.Layout(width='150px'))
tp_start_ = widgets.IntText(value=5, description='TP onset (ms):', layout=widgets.Layout(width='150px'))
vm_jump_ = widgets.IntText(value=10, description='TP Vm jump (mV):', layout=widgets.Layout(width='150px'))
pre_tp_ = widgets.IntText(value=3, description='TP pre window (ms):', layout=widgets.Layout(width='150px'))
fs_ = widgets.IntText(value=25, description='Sampling Freq (kHz):', layout=widgets.Layout(width='150px'))
amp_factor_ = widgets.IntText(value=1, description='Scaler for plots in pA', layout=widgets.Layout(width='150px'))
timepoint_ = widgets.Dropdown(options=['p2', 'p14'], value='p2', description='Timepoint', layout=widgets.Layout(width='150px'))

metadata = [lowpass_freq_, stim_time_, post_stim_, tp_start_, vm_jump_, pre_tp_, fs_, amp_factor_, timepoint_]
metadata_box = widgets.GridBox(metadata, layout=widgets.Layout(grid_template_columns='repeat(2, 160px)'))

In [3]:
data_choose = FileChooser()
data_choose.default_path = os.path.expanduser('~')
data_choose.use_dir_icons = True
data_choose.title = '<b>Select Data</b>'

data_notes_choose = FileChooser()
data_notes_choose.default_path = os.path.expanduser('~')
data_notes_choose.use_dir_icons = True
data_notes_choose.title = '<b>Select Data Notes</b>'

sweep_slider = widgets.IntSlider(value=0, min=0, max=10, step=1, description='Sweep #: ', continuous_update=False)
sweep_left = widgets.Button(icon='angle-left', tooltip='next sweep', layout=widgets.Layout(width='40px'))
sweep_right = widgets.Button(icon='angle-right', tooltip='previous sweep', layout=widgets.Layout(width='40px'))

open_cell = widgets.Button(description='Open Cell', tooltip='open selected cell file with data notes', layout=widgets.Layout(width='80px'))

textout = widgets.Output()
plotout = widgets.Output()
dropout = widgets.Output()

drop_sweep = widgets.Button(description='Drop', tooltip='adds sweep to drop_list', layout=widgets.Layout(width='63px'))
undrop_sweep = widgets.Button(description='Undrop', tooltip='removes sweep from drop_list', layout=widgets.Layout(width='63px'))
save_drops = widgets.Button(icon='save', tooltip='save dropped sweeps', layout=widgets.Layout(width='32px'))

dropout = widgets.Output()

def update_values(values):
    return values


In [4]:
def open_cell_click(button):
    # set metadata from widgets
    lowpass_freq = lowpass_freq_.value
    stim_time = stim_time_.value
    post_stim = post_stim_.value
    tp_start = tp_start_.value
    vm_jump = vm_jump_.value
    pre_tp = pre_tp_.value
    fs = fs_.value
    amp_factor = amp_factor_.value
    timepoint = timepoint_.value

    drop_list = []
    file = data_choose.selected_filename
    file_path = data_choose.selected
    path = data_choose.selected_path
    data_notes = data_notes_choose.selected
    cell = clamp_ephys.workflows.cell(file_path, fs=fs, path_to_data_notes=data_notes, timepoint=timepoint, amp_factor=amp_factor)
    cell.filter_traces(lowpass_freq)
    trace = cell.traces_filtered[0]
    time = np.arange(0, len(trace) / cell.fs, 1 / cell.fs)
    ntraces = len(cell.traces_filtered.columns)
    sweep_slider.max = ntraces - 1
    TOOLS = ['pan, wheel_zoom, crosshair, reset']
    
    textout.clear_output()
    plotout.clear_output()
    dropout.clear_output()
    
    with textout:
        print(f'{file} opened')

    with plotout:
        p = figure(plot_width=800, plot_height=400, title=f'Sweep #: {trace.name}', toolbar_location='above',
                   tools=TOOLS, active_scroll="wheel_zoom")
        p.xaxis.axis_label = 'Time (ms)'
        p.yaxis.axis_label = 'Current (pA)'
        sweep = p.line(time, trace)
        show(p, notebook_handle=True)

    def sweep_change(change):
        with plotout:
            sweep.data_source.data['y'] = cell.traces_filtered[change['new']]
            p.title.text = f'Sweep #: {sweep_slider.value}'
            push_notebook()
    def update_drop_sweep(button):
        sweep_number = sweep_slider.value
        if sweep_number not in drop_list:
            dropout.clear_output()
            drop_list.append(sweep_number)
            drop_list.sort()
            update_string = f'Dropped sweeps: {drop_list}\nsweep {sweep_number} dropped'
            with dropout:
                print(update_string)

    def update_undrop_sweep(button):
        sweep_number = sweep_slider.value
        if sweep_number in drop_list:
            dropout.clear_output()
            drop_list.remove(sweep_number)
            drop_list.sort()
            update_string = f'Dropped sweeps: {drop_list}\nsweep {sweep_number} undropped'
        else:
            dropout.clear_output()
            update_string = f'Dropped sweeps: {drop_list}\nno sweeps undropped'
        with dropout:
            print(update_string)
                

    def save_dropped_sweeps(button):
        drop_path = os.path.join(path, 'dropped_sweeps.csv')
        new_drops = pd.DataFrame({'dropped': [drop_list]}, index=[file])

        if os.path.exists(drop_path) == True:
            dropped_sweeps = pd.read_csv(drop_path, index_col=[0])
            if file in dropped_sweeps.index:
                dropped_sweeps.drop(index=file, inplace=True)

            dropped_sweeps = pd.concat([dropped_sweeps, new_drops])

        else:
            dropped_sweeps = new_drops

        dropped_sweeps.to_csv(drop_path)

        dropout.clear_output()
        update_string = f'Dropped sweeps: {drop_list}\nSaved in {drop_path}'
        
        with dropout:
            print(update_string)
            
    sweep_slider.observe(sweep_change, names='value')
    drop_sweep.on_click(update_drop_sweep)
    undrop_sweep.on_click(update_undrop_sweep)
    save_drops.on_click(save_dropped_sweeps)
    
def sweep_left_click(button):
    if sweep_slider.value > 0:
        sweep_slider.value -= 1

def sweep_right_click(button):
    if sweep_slider.value < sweep_slider.max:
        sweep_slider.value += 1
        


sweep_left.on_click(sweep_left_click)
sweep_right.on_click(sweep_right_click)
open_cell.on_click(open_cell_click)

In [5]:
file_box = widgets.VBox([data_notes_choose, data_choose])
top_box = widgets.HBox([file_box, metadata_box])
open_box = widgets.HBox([open_cell, textout])
sweep_box = widgets.HBox([sweep_slider, sweep_left, sweep_right, drop_sweep, undrop_sweep, save_drops, dropout])
explorer = widgets.VBox([top_box, open_box, sweep_box, plotout])

### Explorer General Use
1. Select a Data Notes file under Select Data Notes
2. Select a Data file of your cell of interest.
3. Make sure the metadata are all set correctly before clicking `Open Cell`. If you need to make changes to the metadata, you'll need click `Open Cell` again. 
4. Click `Open Cell`. The plot of your trace open to the current slider value will open.
5. The tools pan, wheel_zoom, and crosshair are active by default. You can toggle them on/off with the toolbar on the top of the plot.
    - pan: click and drag
    - zoom: scroll wheel; if anywhere on plot, will zoom x and y axes; if on x-axis OR y-axis when scroll, will only scroll that axis.
    - crosshair: pretty obvious
    - reset: click to reset to original view
6. You can control the sweep displayed with the slider or the left right arrows. 
7. There is a `drop_list` created when you open the cell which begins as an empty list. Add the displayed sweep the the `drop_list` by clicking `Drop`. If this was a mistake, remove the displayed sweep from `drop_list` by clikcing `undrop`. 
8. Save `drop_list` to file by clicking the save icon. The saving will be done automatically, and `drop_list` can be read in automatically with code by setting the arg `dropped_sweeps=True` in workflows.cell(). You **MUST** save the `drop_list` before opening a new cell. `drop_list` is always cleared between openings.
    - NOTE: the save function will automatically overwrite whatever previously saved dropped sweeps you have for that file. This is useful if you want to redo your drop criteria. This is harmful in the case where you did not want to overwrite your data. Use carefully.  


In [6]:
explorer

In [7]:
data_choose.selected_filename

NameError: name 'drop_list' is not defined